In [1]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 9.8 MB/s eta 0:00:00
167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 43.3s167.7 MiB [] 0% 32.3s167.7 MiB [] 0% 23.9s167.7 MiB [] 0% 13.4s167.7 MiB [] 0% 11.8s167.7 MiB [] 1% 9.3s167.7 MiB [] 1% 7.5s167.7 MiB [] 1% 6.9s167.7 MiB [] 2% 6.0s167.7 MiB [] 2% 5.6s167.7 MiB [] 3% 5.1s167.7 MiB [] 3% 4.9s167.7 MiB [] 4% 4.6s167.7 MiB [] 4% 4.4s167.7 MiB [] 5% 4.3s167.7 MiB [] 5% 4.2s167.7 MiB [] 6% 4.5s167.7 MiB [] 6% 4.6s167.7 MiB [] 6% 4.5s167.7 MiB [] 7% 4.4s167.7 MiB [] 8% 4.4s167.7 MiB [] 8% 4.3s167.7 MiB [] 9% 4.3s167.7 MiB [] 9% 4.4s167.7 MiB [] 10% 4.3s167.7 MiB [] 10% 4.2s167.7 MiB [] 11% 4.2s167.7 MiB [] 11% 4.1s167.7 MiB [] 12% 4.0s167.7 MiB [] 13% 3.9s167.7 MiB [] 14% 3.8s167.7 MiB [] 15% 3.7s167.7 MiB [] 16% 3.6s167.7 MiB [] 16% 3.5s167.7 MiB [] 17% 3.4s167.7 MiB [] 18% 3.3s167.7 MiB [] 19% 3.1s167.7 MiB [] 20% 3.1s167.7 MiB [] 21% 3.0s167.7 MiB [] 21% 2.9s167.7 MiB [] 22% 2.9s167.7 MiB [] 23% 2.8s167.7 MiB [] 24% 2.7s167.7 MiB [] 25% 2

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [5]:
import pandas as pd
from playwright.async_api import async_playwright
import asyncio

async def scrape_almeria():
    url = "https://www.booking.com/searchresults.es.html?lang=es&selected_currency=EUR&dest_id=1363&dest_type=region&ac_langcode=es&checkin=2025-05-07&checkout=2025-05-08&nflt=ht_id%3D204&shw_aparth=0"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url, timeout=60000)

        # Aceptar cookies si aparece
        try:
            await page.click("button:has-text('Aceptar')", timeout=5000)
            print("Cookies aceptadas.")
        except:
            print("No se mostró banner de cookies.")

        # Hacer scroll lento
        for _ in range(15):
            await page.mouse.wheel(0, 1500)
            await page.wait_for_timeout(1000)

        # Esperar por los resultados
        try:
            await page.wait_for_selector('[data-testid="property-card"]', timeout=30000)
        except:
            print("❌ No se encontró ningún hotel. Posible bloqueo o la página está vacía.")
            content = await page.content()
            print("Fragmento del HTML para diagnóstico:\n", content[:2000])
            await browser.close()
            return

        hotels = await page.locator('[data-testid="property-card"]').all()
        print(f"✅ Se encontraron {len(hotels)} hoteles.")

        hotels_data = []

        for hotel in hotels:
            data = {}
            try:
                data['Hotel'] = await hotel.locator('[data-testid="title"]').inner_text()
            except:
                data['Hotel'] = ''
            try:
                data['Precio'] = await hotel.locator('[data-testid="price-and-discounted-price"]').inner_text()
            except:
                data['Precio'] = ''
            try:
                score = await hotel.locator('[data-testid="review-score"] div').all_inner_texts()
                data['rating'] = score[0] if len(score) > 0 else ''
                data['review_summary'] = score[1] if len(score) > 1 else ''
                data['review_count'] = score[2].split()[0] if len(score) > 2 else ''
            except:
                data['rating'] = ''
                data['review_summary'] = ''
                data['review_count'] = ''
            hotels_data.append(data)

        df = pd.DataFrame(hotels_data)
        df.to_csv("hotels_almeria.csv", index=False)
        print("✅ Archivo 'hotels_almeria.csv' guardado con éxito.")
        await browser.close()

# Ejecutar en Colab
await scrape_almeria()



No se mostró banner de cookies.
✅ Se encontraron 75 hoteles.
✅ Archivo 'hotels_almeria.csv' guardado con éxito.
